In [1]:
from pykiwoom.kiwoom import *
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time, warnings, pickle
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

In [2]:
# login
kiwoom = Kiwoom() # allocation
print('login..')
kiwoom.CommConnect(block=True) # waiting to login
print('login..complete')
state = kiwoom.GetConnectState()
if state == 0:
    print('not connected')
elif state == 1:
    print('status : connected')

login..
login..complete
status : connected


In [3]:
kiwoom.DisconnectRealData(screen=1234)

In [4]:
KOR = pd.read_csv('./Data_portdoctor/KORandWTI_raw.csv',  encoding='UTF-8', header=0, index_col=0)
KOR_monthly = pd.read_csv('./Data_portdoctor/KOR_monthly_raw.csv',  encoding='UTF-8', header=0, index_col=0)
KOR['spread1'] =  KOR['bond_3y'] - KOR['msbond_91d']
KOR['spread2'] =  KOR['bond_10y'] - KOR['bond_3y']
KOR.index = pd.to_datetime(KOR.index)
KOR_monthly.index = pd.to_datetime(KOR_monthly.index)
KOR_sub = KOR.resample('M').mean()
KOR_monthly_sub = KOR_monthly.resample('M').mean()

KOR_monthly = KOR_monthly.astype(float).apply(lambda x: np.log(x)).diff(1).iloc[1:]
KOR_interest = KOR[['call_1d', 'spread1', 'spread2']].applymap(lambda x: float(x) if type(x)!=np.NaN else x)
KOR_interest = KOR_interest.diff(1).iloc[1:]   #pct_change로 바꾸기 위함
KOR_not_interest = KOR[['KOSPI', 'KOSDAQ', 'krwusd', 'news_sentiment', 'WTI']].astype(float).apply(lambda x: np.log(x)).diff(1).iloc[1:]       
KOR = pd.merge(KOR_interest, KOR_not_interest, left_index=True, right_index=True)

with open('./Data_portdoctor/stock_return_monthly.pkl', 'rb') as f:   #stock return은 monthly수익률임 (1+등락률의 곱에 1빼줌)
    returns = pickle.load(f)
KOR_return = pd.DataFrame(returns)
KOR_return.index = pd.to_datetime(KOR_return.index)

In [5]:
KOR_return[-1:].dropna(1).columns

Index(['001460', '001465', '035760', '014820', '014825', '006200', '030210',
       '017940', '006360', '010120',
       ...
       '328130', '424760', '341170', '365340', '397030', '418210', '368600',
       '126720', '107600', '430700'],
      dtype='object', length=2640)

In [15]:
stock_list = KOR_return[-1:].dropna(1).columns[31:70]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        time.sleep(1)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(1)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

In [16]:
result

,017670,016600,024070,040610,000860,001390,039240,002100,012320,009450,...,030610,007690,005320,001140,043650,006050,000680,002720,028300,007720
일자,,,,,,,,,,,,,,,,,,,,,
20220829,4230,46,4,-63,-1,474,3,89,70,309,...,47,-150,-2,-96,56,459,45,-8,4758,4
20220826,9579,44,0,-48,32,-453,99,20,12,665,...,-7,-557,37,-160,164,-28,-53,16,-7107,3
20220825,-13855,-59,1,-39,25,108,17,-189,-10,55,...,7,31,-70,-7,148,-71,-58,-89,-3917,-7
20220824,1608,84,1,8,15,-100,-347,11,12,268,...,165,-642,-40,3,482,3,24,-12,1367,-31
20220823,-1031,73,3,-5,15,-491,-208,106,3,156,...,50,-211,33,76,149,-15,-55,30,187,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200826,7672,14,-11,-40,-13,369,5,1196,5,-116,...,43,-26,-604,-187,-207,128,4,0,9123,11
20200825,-11149,-18,0,-32,847,159,-24,496,-6,-468,...,6,-131,-110,34,-57,-103,-1,-576,-2574,-9
20200824,4616,3,1,-5,45,-30,-1,8,13,-555,...,-37,-11,269,47,-5,20,13,1252,-6827,-10


In [17]:
net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

In [18]:
net_order_indiv.to_csv('개인투자자순매수31-70.csv')

In [19]:
stock_list = KOR_return[-1:].dropna(1).columns[71:110]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        time.sleep(1)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(1)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수71-110.csv')

In [20]:
stock_list = KOR_return[-1:].dropna(1).columns[111:150]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        time.sleep(1)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(1)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수111-150.csv')

In [21]:
stock_list = KOR_return[-1:].dropna(1).columns[151:190]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        time.sleep(1)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(1)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수151-190.csv')

In [22]:
stock_list = KOR_return[-1:].dropna(1).columns[191:230]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        time.sleep(1)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(1)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수191-230.csv')

In [7]:
stock_list = KOR_return[-1:].dropna(1).columns[231:270]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        time.sleep(1)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(1)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수231-270.csv')

In [8]:
stock_list = KOR_return[-1:].dropna(1).columns[271:310]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        time.sleep(1)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(1)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수271-310.csv')

In [9]:
stock_list = KOR_return[-1:].dropna(1).columns[311:350]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(3.7)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수311-350.csv')

In [10]:
stock_list = KOR_return[-1:].dropna(1).columns[351:390]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(3.7)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수351-390.csv')

In [11]:
stock_list = KOR_return[-1:].dropna(1).columns[391:430]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(3.7)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수391-430.csv')

In [6]:
stock_list = KOR_return[-1:].dropna(1).columns[431:470]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(3.7)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수431-470.csv')

In [7]:
stock_list = KOR_return[-1:].dropna(1).columns[471:500]
result = pd.DataFrame()
for stock in stock_list:
    dfs = kiwoom.block_request("opt10059",
                            일자=datetime.today().strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,  #금액
                            매매구분=0,  #순매수
                            단위구분=1000,  #1000주
                            output="종목별투자자기관별요청",
                            next=0)

    while dfs.shape[0]<500:
        df = kiwoom.block_request("opt10059",
                            일자=(datetime.strptime(dfs['일자'].iloc[-1], '%Y%m%d') - relativedelta(days=1)).strftime('%Y%m%d'),
                            종목코드=stock,
                            금액수량구분=1,
                            매매구분=0,
                            단위구분=1000,
                            output="종목별투자자기관별요청",
                            next=0)
        dfs = pd.concat([dfs, df])

    dfs.set_index(dfs['일자'], inplace=True)
    time.sleep(3.7)
    result = pd.concat([result, dfs[['개인투자자']]], axis=1)
result.columns = stock_list

net_order_indiv = result.copy()
net_order_indiv = net_order_indiv.astype(float)
net_order_indiv.index = pd.to_datetime(net_order_indiv.index)
net_order_indiv = net_order_indiv.resample('M').sum()

net_order_indiv.to_csv('개인투자자순매수471-500.csv')